<a href="https://colab.research.google.com/github/niharpalem/resume_easz/blob/main/nb/Llama3.2_(1B_and_3B)-Conversational.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### News

**[NEW] We've fixed many bugs in Phi-4** which greatly increases Phi-4's accuracy. See our [blogpost](https://unsloth.ai/blog/phi4)

[NEW] You can view all Phi-4 model uploads with our bug fixes including [dynamic 4-bit quants](https://unsloth.ai/blog/dynamic-4bit), GGUF & more [here](https://huggingface.co/collections/unsloth/phi-4-all-versions-677eecf93784e61afe762afa)

[NEW] As of Novemeber 2024, Unsloth now supports [vision finetuning](https://unsloth.ai/blog/vision)!


### Installation

In [1]:
%%capture
# Normally using pip install unsloth is enough

# Temporarily as of Jan 31st 2025, Colab has some issues with Pytorch
# Using pip install unsloth will take 3 minutes, whilst the below takes <1 minute:
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth

### Unsloth

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",

    "unsloth/Llama-3.3-70B-Instruct-bnb-4bit" # NEW! Llama 3.3 70B!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-bnb-4bit", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.8: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.1.8 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [6]:
PROMPT_TEMPLATE = """
<|begin_of_text|><|start_header_id|>user<|end_header_id|>
Extract the following information from the job description:
- Job title
- Required skills
- Desired skills
- Responsibilities
- Requirements
- Experience level

Job Description:
{description}

Format the output as JSON.<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""


In [7]:
import json

def extract_job_info(description):
    prompt = PROMPT_TEMPLATE.format(description=description)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=512, temperature=0.1)

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    try:
        # Attempt to parse the JSON response
        extracted_info = json.loads(response)
    except json.JSONDecodeError:
        # If parsing fails, return a default structure
        extracted_info = {
            "job_title": "",
            "required_skills": [],
            "desired_skills": [],
            "responsibilities": [],
            "requirements": [],
            "experience_level": ""
        }

    return extracted_info


In [8]:
import pandas as pd

In [9]:
data=pd.read_feather("/content/merged_job_data.feather")

In [10]:
data=data[data['description'].notnull()]

In [11]:
data.reset_index(drop=True,inplace=True)

In [12]:
data=data[['title','description']]

In [13]:
data.title.value_counts().sort_values(ascending=False).head(8)

,count
title,
Data Scientist,304
Business Analyst,277
Data Analyst,253
Data Engineer,215
Senior Data Engineer,186
Senior Data Scientist,183
Financial Analyst,163
Software Engineer,108


In [14]:
#Get the top 8 most frequent titles
top_titles = data.title.value_counts().sort_values(ascending=False).head(8).index


In [15]:
filtered_df = data[data.title.isin(top_titles)]

In [16]:
filtered_df.reset_index(drop=True,inplace=True)

In [17]:
# Define a function to clean the description
def clean_description(text):
    # Remove newline characters
    text = text.replace('\n', ' ')
    # Remove other unwanted symbols (example: **, --, etc.)
    text = text.replace('**', '')
    text = text.replace('--', '')
    # Add more replacements as needed
    return text

# Apply the cleaning function to the description column
filtered_df['description'] = filtered_df['description'].apply(clean_description)

# Display the cleaned DataFrame
print(filtered_df)

                      title                                        description
0         Software Engineer  Job Description Software Engineer Sacramento, ...
1      Senior Data Engineer  OVERVIEW OF THE COMPANY Fox Corporation Under ...
2     Senior Data Scientist  Overview    We are seeking a highly motivated ...
3              Data Analyst  Job Summary:     This individual contributor i...
4            Data Scientist  Diamond Foundry Inc. is solving the thermal li...
...                     ...                                                ...
1684           Data Analyst  This is an in person role in our Raleigh, Nort...
1685           Data Analyst  Job Overview   We are seeking a detail-oriente...
1686           Data Analyst  Job Overview   We are seeking a detail-oriente...
1687      Financial Analyst  Kipu is bettering the behavioral health landsc...
1688           Data Analyst  Position Overview We are seeking a Data Analys...

[1689 rows x 2 columns]


<ipython-input-17-631dce896eb2>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['description'] = filtered_df['description'].apply(clean_description)


In [18]:
filtered_df

,title,description
0,Software Engineer,"Job Description Software Engineer Sacramento, ..."
1,Senior Data Engineer,OVERVIEW OF THE COMPANY Fox Corporation Under ...
2,Senior Data Scientist,Overview We are seeking a highly motivated ...
3,Data Analyst,Job Summary: This individual contributor i...
4,Data Scientist,Diamond Foundry Inc. is solving the thermal li...
...,...,...
1684,Data Analyst,"This is an in person role in our Raleigh, Nort..."
1685,Data Analyst,Job Overview We are seeking a detail-oriente...
1686,Data Analyst,Job Overview We are seeking a detail-oriente...
1687,Financial Analyst,Kipu is bettering the behavioral health landsc...


In [20]:
model = FastLanguageModel.for_inference(model)

In [26]:
def extract_job_info(description):
    prompt = PROMPT_TEMPLATE.format(description=description)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=512, temperature=0.5)

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    try:
        # Attempt to parse the JSON response
        extracted_info = json.loads(response)
    except json.JSONDecodeError:
        # If parsing fails, return a default structure
        extracted_info = {
            "job_title": "",
            "required_skills": [],
            "desired_skills": [],
            "responsibilities": [],
            "requirements": [],
            "experience_level": ""
        }

    return extracted_info


In [ ]:
filtered_df['extracted_info'] = filtered_df['description'].apply(extract_job_info)


In [23]:
filtered_df

,title,description,extracted_info
0,Software Engineer,"Job Description Software Engineer Sacramento, ...","{'job_title': '', 'required_skills': [], 'desi..."
1,Senior Data Engineer,OVERVIEW OF THE COMPANY Fox Corporation Under ...,"{'job_title': '', 'required_skills': [], 'desi..."
2,Senior Data Scientist,Overview We are seeking a highly motivated ...,"{'job_title': '', 'required_skills': [], 'desi..."
3,Data Analyst,Job Summary: This individual contributor i...,"{'job_title': '', 'required_skills': [], 'desi..."
4,Data Scientist,Diamond Foundry Inc. is solving the thermal li...,"{'job_title': '', 'required_skills': [], 'desi..."
...,...,...,...
1684,Data Analyst,"This is an in person role in our Raleigh, Nort...","{'job_title': '', 'required_skills': [], 'desi..."
1685,Data Analyst,Job Overview We are seeking a detail-oriente...,"{'job_title': '', 'required_skills': [], 'desi..."
1686,Data Analyst,Job Overview We are seeking a detail-oriente...,"{'job_title': '', 'required_skills': [], 'desi..."
1687,Financial Analyst,Kipu is bettering the behavioral health landsc...,"{'job_title': '', 'required_skills': [], 'desi..."


In [24]:
filtered_df['job_title'] = filtered_df['extracted_info'].apply(lambda x: x.get('job_title', ''))
filtered_df['required_skills'] = filtered_df['extracted_info'].apply(lambda x: x.get('required_skills', []))
filtered_df['desired_skills'] = filtered_df['extracted_info'].apply(lambda x: x.get('desired_skills', []))
filtered_df['responsibilities'] = filtered_df['extracted_info'].apply(lambda x: x.get('responsibilities', []))
filtered_df['requirements'] = filtered_df['extracted_info'].apply(lambda x: x.get('requirements', []))
filtered_df['experience_level'] = filtered_df['extracted_info'].apply(lambda x: x.get('experience_level', ''))


<ipython-input-24-f512927bcaf5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['job_title'] = filtered_df['extracted_info'].apply(lambda x: x.get('job_title', ''))
<ipython-input-24-f512927bcaf5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['required_skills'] = filtered_df['extracted_info'].apply(lambda x: x.get('required_skills', []))
<ipython-input-24-f512927bcaf5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

In [25]:
filtered_df

,title,description,extracted_info,job_title,required_skills,desired_skills,responsibilities,requirements,experience_level
0,Software Engineer,"Job Description Software Engineer Sacramento, ...","{'job_title': '', 'required_skills': [], 'desi...",,[],[],[],[],
1,Senior Data Engineer,OVERVIEW OF THE COMPANY Fox Corporation Under ...,"{'job_title': '', 'required_skills': [], 'desi...",,[],[],[],[],
2,Senior Data Scientist,Overview We are seeking a highly motivated ...,"{'job_title': '', 'required_skills': [], 'desi...",,[],[],[],[],
3,Data Analyst,Job Summary: This individual contributor i...,"{'job_title': '', 'required_skills': [], 'desi...",,[],[],[],[],
4,Data Scientist,Diamond Foundry Inc. is solving the thermal li...,"{'job_title': '', 'required_skills': [], 'desi...",,[],[],[],[],
...,...,...,...,...,...,...,...,...,...
1684,Data Analyst,"This is an in person role in our Raleigh, Nort...","{'job_title': '', 'required_skills': [], 'desi...",,[],[],[],[],
1685,Data Analyst,Job Overview We are seeking a detail-oriente...,"{'job_title': '', 'required_skills': [], 'desi...",,[],[],[],[],
1686,Data Analyst,Job Overview We are seeking a detail-oriente...,"{'job_title': '', 'required_skills': [], 'desi...",,[],[],[],[],
1687,Financial Analyst,Kipu is bettering the behavioral health landsc...,"{'job_title': '', 'required_skills': [], 'desi...",,[],[],[],[],
